In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data_dict = pickle.load(open("final_project_dataset.pkl", "r") )

In [3]:
pdata = pd.DataFrame.from_dict(data_dict, orient='index')

In [4]:
pdata = pdata.replace('NaN', np.nan, regex=True)

In [5]:
# [print(x) for x in pdata['email_address']]
# pdata['test'] = pdata['email_address'] == np.nan
temp = []
for el in pdata['email_address'].values:
    if type(el) == float:
        temp.append(False)
    else:
        temp.append(True)
temp
pdata['has_email_data'] = temp
pdata['total_navalue_count'] = pdata.isnull().sum(axis=1)
pdata['sal_navalue_count'] = pdata[['salary', 'deferral_payments', 'long_term_incentive', 'total_payments', 'bonus', 'expenses', 'loan_advances', 'other', 'director_fees', 'deferred_income']].isnull().sum(axis=1)

In [6]:
# count NA by column
# pdata.isnull().sum()
# count NA by row
# pdata.isnull().sum(axis=1)
# subset only email fields
# pdata[['to_messages', 'shared_receipt_with_poi', 'from_messages', 'from_this_person_to_poi', 'from_poi_to_this_person']]
# subset only rows with email data and examine ratio of pois in that group
# email_only = pdata.dropna(subset=['to_messages'])
# without = email_only['poi'].value_counts()
# print('without Nan email data ratio: ', without[1]/float(without[0]))
# print(email_only.shape)
# wit = pdata['poi'].value_counts()
# print('wit email Nan data ratio: ', wit[1]/float(wit[0]))
# print(pdata.shape)
pd.crosstab(pdata['poi'], pdata['has_email_data'])

has_email_data,False,True
poi,,
False,35,93
True,0,18


In [7]:
pdata['navalue_count'] = pdata.isnull().sum(axis=1)

In [6]:
# # # Clean email only data
# email_only = email_only.drop('email_address', 1)
# email_only = email_only.replace(np.nan, 0, regex=True)
# email_only = email_only[email_only.index != 'TOTAL']
# Clean pdata
# pdata = pdata.drop(['to_messages', 'shared_receipt_with_poi', 'from_messages', 'from_this_person_to_poi', 'from_poi_to_this_person', 'email_address'], 1)
# pdata = pdata.replace(np.nan, 0, regex=True)
# pdata = pdata[pdata.index != 'TOTAL']
pdata = pdata.drop('email_address', 1)
pdata = pdata.replace(np.nan, 0, regex=True)
pdata = pdata[pdata.index != 'TOTAL']

In [7]:
pdata['short_gain'] = pdata['exercised_stock_options'] - pdata['long_term_incentive']
# gr = pdata.groupby(['poi'])
# gr['short_gain'].median()
# gr['exercised_stock_options'].median()

In [10]:
# 'FREVERT MARK A', 'LAY KENNETH L',
# outies = ['FREVERT MARK A', 'LAY KENNETH L', 'SKILLING JEFFREY K']
# pdata = pdata.drop(outies)

In [11]:
# Approach 1
# Use only rows that contain email data
temp = pdata['salary'] + pdata['bonus'] + pdata['long_term_incentive'] + \
    pdata['deferred_income'] + pdata['deferral_payments'] + pdata['loan_advances'] + \
    pdata['other'] + pdata['expenses'] + pdata['director_fees']
temp == pdata['total_payments']
# BELFER ROBERT and BHATNAGAR SANJAY

ALLEN PHILLIP K                   True
BADUM JAMES P                     True
BANNANTINE JAMES M                True
BAXTER JOHN C                     True
BAY FRANKLIN R                    True
BAZELIDES PHILIP J                True
BECK SALLY W                      True
BELDEN TIMOTHY N                  True
BELFER ROBERT                    False
BERBERIAN DAVID                   True
BERGSIEKER RICHARD P              True
BHATNAGAR SANJAY                 False
BIBI PHILIPPE A                   True
BLACHMAN JEREMY M                 True
BLAKE JR. NORMAN P                True
BOWEN JR RAYMOND M                True
BROWN MICHAEL                     True
BUCHANAN HAROLD G                 True
BUTTS ROBERT H                    True
BUY RICHARD B                     True
CALGER CHRISTOPHER F              True
CARTER REBECCA C                  True
CAUSEY RICHARD A                  True
CHAN RONNIE                       True
CHRISTODOULOU DIOMEDES            True
CLINE KENNETH W          

In [8]:
pdata = pdata[pdata.index != 'LAY KENNETH L']
pdata = pdata[pdata.index != 'BELFER ROBERT']
pdata = pdata[pdata.index != 'BHATNAGAR SANJAY']

In [9]:
temp = pdata['exercised_stock_options'] + pdata['restricted_stock'] + pdata['restricted_stock_deferred']
len(pdata) -(temp == pdata['total_stock_value']).sum()
# excellent. indicates no mismatch...

0

In [10]:
np.set_printoptions(precision=2)

In [11]:
pdata['sal_ratio'] = (pdata['bonus'] + pdata['long_term_incentive'] + pdata['expenses']) / pdata['total_payments']
pdata['stock_ratio'] = pdata['restricted_stock'] / pdata['total_stock_value']
pdata['em_ratio'] = (pdata['shared_receipt_with_poi'] + pdata['from_this_person_to_poi'] + pdata['from_poi_to_this_person'] ) / (pdata['to_messages'] + pdata['from_messages'])
pdata['total_total'] = pdata['total_payments'] + pdata['total_stock_value']
pdata['from_ratio'] = pdata['from_this_person_to_poi'] / pdata['from_messages']
pdata['sent_ratio'] = (pdata['from_poi_to_this_person'] + pdata['shared_receipt_with_poi']) / pdata['to_messages']

In [16]:
# pdata = pdata.drop('total_payments', 1)
# pdata = pdata.drop('total_stock_value', 1)

In [12]:
# edata = email_only.T.to_dict()
pdata = pdata.replace(np.nan, 0, regex=True)
pdata = pdata.replace(np.inf, 0, regex=True)
pdata = pdata.replace(-np.inf, 0, regex=True)
pdict = pdata.T.to_dict()

In [13]:
import sys
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit

In [19]:
features_list = ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments',
       'exercised_stock_options', 'bonus', 'restricted_stock',
       'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value',
       'expenses', 'from_messages', 'other', 'from_this_person_to_poi',
       'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'from_ratio', 'sent_ratio',
       'sal_ratio', 'stock_ratio', 'em_ratio', 'has_email_data', 'total_navalue_count','sal_navalue_count', 'total_total', 'short_gain']
print(len(features_list))

29


In [20]:
my_dataset = pdict
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [22]:
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [24]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

x_new = SelectKBest(k=12)
x_new.fit_transform(features_train, labels_train)
#model = SVC(kernel='linear', C=6).fit(features_train, labels_train)
#print(model)
for i in range(len(x_new.get_support())):
    if x_new.get_support()[i]:
        print(features_list[i+1])
# features_list[x_new.get_support()]
#accuracy_score(labels_test, model.predict(features_test))
# print(x_new.scores_)
# # tups = {}
# # for i in range(27):
# #     tups[features_list[i+1]] = x_new.scores_[i]
# # for w in sorted(tups, key=tups.get, reverse=True):
# #   print w, tups[w]

salary
total_payments
exercised_stock_options
bonus
deferred_income
from_ratio
sent_ratio
sal_ratio
em_ratio
total_navalue_count
sal_navalue_count
total_total


In [77]:
from sklearn.feature_selection import SelectPercentile, f_classif
for i in range(10):
    x_new = SelectKBest(k=i+1)
    x_new.fit_transform(features_train, labels_train)
    print(x_new.get_params())
    model = SVC(kernel='linear', C=i+1).fit(features_train, labels_train)
    pred = model.predict(features_test)
    print(i, accuracy_score(labels_test, pred), precision_score(labels_test, pred), recall_score(labels_test, pred))
    for i in range(len(x_new.get_support())):
        if x_new.get_support()[i]:
            print(features_list[i+1])

{'k': 1, 'score_func': <function f_classif at 0x1086cab18>}
(0, 0.90697674418604646, 0.5, 0.25)
bonus
{'k': 2, 'score_func': <function f_classif at 0x1086cab18>}
(1, 0.90697674418604646, 0.5, 0.25)
salary
bonus
{'k': 3, 'score_func': <function f_classif at 0x1086cab18>}
(2, 0.88372093023255816, 0.33333333333333331, 0.25)
salary
bonus
total_stock_value
{'k': 4, 'score_func': <function f_classif at 0x1086cab18>}
(3, 0.88372093023255816, 0.33333333333333331, 0.25)
salary
exercised_stock_options
bonus
total_stock_value
{'k': 5, 'score_func': <function f_classif at 0x1086cab18>}
(4, 0.86046511627906974, 0.25, 0.25)
salary
exercised_stock_options
bonus
total_stock_value
from_ratio
{'k': 6, 'score_func': <function f_classif at 0x1086cab18>}
(5, 0.86046511627906974, 0.25, 0.25)
salary
exercised_stock_options
bonus
total_stock_value
from_ratio
short_gain
{'k': 7, 'score_func': <function f_classif at 0x1086cab18>}
(6, 0.86046511627906974, 0.25, 0.25)
salary
exercised_stock_options
bonus
total_st

In [73]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

# pca = PCA(n_components=2)
selection = SelectKBest(k=10)
# combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
# X_features = combined_features.fit(features_train, labels_train).transform(features_train)
svm = SVC(kernel="linear")
pipeline = Pipeline([('univ_select', selection), ("svm", svm)])
# param_grid = dict(features__pca__n_components=[1, 2, 3],
#                   features__univ_select__k=[1, 2, 3],
#                   svm__C=[0.1, 1, 5, 10])
# grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
# grid_search.fit(features_train, labels_train)
model = pipeline.fit(features_train, labels_train)
# print(grid_search.best_estimator_)
# pred = grid_search.best_estimator_.predict(features_test)
pred = model.predict(features_test)
print(pred)
accuracy_score(labels_test, pred)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


0.90697674418604646

In [ ]:
from sklearn import cross_validation

features_list = ['poi', 'total_total', 'from_ratio']

data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

scaler = MinMaxScaler()
features = scaler.fit_transform(features)    

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

model = SVC(kernel='linear', C=5).fit(features_train, labels_train)

print('this is the model:')
print(model)

scores = cross_validation.cross_val_score(model, features, labels, cv=10)

print('these are the xval scores:')
print(scores)

predicted = cross_validation.cross_val_predict(model, features, labels, cv=10)

print('accuracy:', accuracy_score(labels, predicted))
print('recall: ', recall_score(labels, predicted))

In [75]:
# features_list = ['poi', 'bonus', 'salary', 'total_stock_value', 'deferred_income',
#                  'short_gain', 'total_total', 'sal_ratio',
#                  'shared_receipt_with_poi', 'restricted_stock', 'em_ratio', 'other']

# features_list = ['poi', 'salary', 'bonus', 'total_stock_value', 'exercised_stock_options',
#                  'from_ratio', 'short_gain', 'deferred_income', 'total_total', 'sal_ratio',
#                  'total_payments']

features_list = ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments',
       'exercised_stock_options', 'bonus', 'restricted_stock',
       'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value',
       'expenses', 'from_messages', 'other', 'from_this_person_to_poi',
       'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'from_ratio', 'sent_ratio',
       'sal_ratio', 'stock_ratio', 'em_ratio', 'has_email_data', 'navalue_count', 'total_total', 'short_gain']

#                  'exercised_stock_options', 'bonus', 'total_stock_value', 
#                  'expenses', 'other', 'shared_receipt_with_poi', 'salary']
#                  'navalue_count', 'shared_receipt_with_poi', 'stock_ratio', 'deferred_income', 'salary',
#                  'from_poi_to_this_person', 'from_this_person_to_poi', 'sal_ratio', 'long_term_incentive',
#'sal_ratio', 'em_ratio'

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler

data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

scaler = MinMaxScaler()
features = scaler.fit_transform(features)    

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=10)
rfc.fit(features_train, labels_train)

pred = rfc.predict(features_test)
print(pred)

print(accuracy_score(labels_test, pred))
print(precision_score(labels_test, pred))
print(recall_score(labels_test, pred))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
0.906976744186
0.5
0.25


In [26]:
for i in range(len(rfc.feature_importances_)):
    print(features_list[i+1], ": ", rfc.feature_importances_[i])

('salary', ': ', 0.029018184001189877)
('to_messages', ': ', 0.023409306048003287)
('deferral_payments', ': ', 0.0049931521507208965)
('total_payments', ': ', 0.034898477060406614)
('exercised_stock_options', ': ', 0.071402652691724036)
('bonus', ': ', 0.074083535054793537)
('restricted_stock', ': ', 0.049646367292265514)
('shared_receipt_with_poi', ': ', 0.035370213416170415)
('restricted_stock_deferred', ': ', 0.0)
('total_stock_value', ': ', 0.042895905516350939)
('expenses', ': ', 0.046640237897074756)
('from_messages', ': ', 0.005435517893799903)
('other', ': ', 0.10019571125026454)
('from_this_person_to_poi', ': ', 0.023254399128601388)
('director_fees', ': ', 0.0)
('deferred_income', ': ', 0.032036870067940731)
('long_term_incentive', ': ', 0.050550767791367822)
('from_poi_to_this_person', ': ', 0.014440498836177207)
('from_ratio', ': ', 0.0789018050661702)
('sent_ratio', ': ', 0.0309714436957562)
('sal_ratio', ': ', 0.027447181626463191)
('stock_ratio', ': ', 0.0235568685508385

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.decomposition import RandomizedPCA
from sklearn.pipeline import Pipeline

accuracy = []
precision = []
recall = []
f1 = []

for i in range(1):
    
    data = featureFormat(my_dataset, features_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)    

    features_train, features_test, labels_train, labels_test = \
        train_test_split(features, labels, test_size=0.3, random_state=42)
        
    pca = RandomizedPCA(n_components=1,whiten=True)#.fit(features_train)
    rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=10)
    pipe = Pipeline(steps=[('pca', pca), ('rfc', rfc)])
    pipe.fit(features_train, labels_train)
    pred = pipe.predict(features_test)    

#     rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=10)
#     rfc.fit(features_train, labels_train)
#     pred = rfc.predict(features_test)
    
    accuracy.append(accuracy_score(labels_test, pred))
    precision.append(precision_score(labels_test, pred))
    recall.append(recall_score(labels_test, pred))
    f1.append(f1_score(labels_test, pred))
    
print('Accuracy: ', np.mean(accuracy))
print('Precision: ', np.mean(precision))
print('Recall: ', np.mean(recall))
print('F1: ', np.mean(f1))


# ('Accuracy: ', 0.90697674418604646)
# ('Precision: ', 0.5)
# ('Recall: ', 0.25)
# ('F1: ', 0.33333333333333331)

('Accuracy: ', 0.90697674418604646)
('Precision: ', 0.5)
('Recall: ', 0.25)
('F1: ', 0.33333333333333331)


In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.decomposition import RandomizedPCA
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler

accuracy = []
precision = []
recall = []
f1 = []

print(features_list)

for i in range(500):
    
    data = featureFormat(my_dataset, features_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    
#     scaler = MinMaxScaler()
    scaler = StandardScaler()
    features = scaler.fit_transform(features)    

    features_train, features_test, labels_train, labels_test = \
        train_test_split(features, labels, test_size=0.3, random_state=42)
    
#     pca = RandomizedPCA(n_components=2,whiten=True)#.fit(features_train)
    
#     features_train_pca = pca.transform(features_train)
#     features_test_pca = pca.transform(features_test)
#     pca_score = pca.explained_variance_ratio_
#     print pca_score
    
    svm = SVC(kernel='linear', C=5.0)
    svm.fit(features_train, labels_train)

#     pipe = Pipeline(steps=[('pca', pca), ('svm', svm)])

#     pipe.fit(features_train, labels_train)

#     pred = pipe.predict(features_test)

    pred = svm.predict(features_test)

    accuracy.append(accuracy_score(labels_test, pred))
    precision.append(precision_score(labels_test, pred))
    recall.append(recall_score(labels_test, pred))
    f1.append(f1_score(labels_test, pred))
    
print('Accuracy: ', np.mean(accuracy))
print('Precision: ', np.mean(precision))
print('Recall: ', np.mean(recall))
print('F1: ', np.mean(f1))

# ('Accuracy: ', 0.90697674418604679)
# ('Precision: ', 0.66666666666666685)
# ('Recall: ', 0.39999999999999991)
# ('F1: ', 0.5)

['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'from_ratio', 'sent_ratio', 'sal_ratio', 'stock_ratio', 'em_ratio', 'has_email_data', 'navalue_count', 'total_total', 'short_gain']
('Accuracy: ', 0.90697674418604657)
('Precision: ', 0.5)
('Recall: ', 0.5)
('F1: ', 0.5)


In [227]:
from sklearn.neighbors import KNeighborsClassifier

accuracy = []
precision = []
recall = []
f1 = []

for i in range(100):
    
    data = featureFormat(my_dataset, features_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)    

    features_train, features_test, labels_train, labels_test = \
        train_test_split(features, labels, test_size=0.3, random_state=42)
    
    pca = RandomizedPCA(n_components=2,whiten=True).fit(features_train)
    
    features_train_pca = pca.transform(features_train)
    features_test_pca = pca.transform(features_test)
#     pca_score = pca.explained_variance_ratio_
#     print pca_score
    
    neigh = KNeighborsClassifier(n_neighbors=2, p=1, weights='distance')
    neigh.fit(features_train_pca, labels_train)

#     pipe = Pipeline(steps=[('pca', pca), ('svm', svm)])

#     pipe.fit(features_train, labels_train)

#     pred = pipe.predict(features_test)

    #print(neigh)

    pred = neigh.predict(features_test_pca)
    
    #print(neigh)

    accuracy.append(accuracy_score(labels_test, pred))
    precision.append(precision_score(labels_test, pred))
    recall.append(recall_score(labels_test, pred))
    f1.append(f1_score(labels_test, pred))
    
print('Accuracy: ', np.mean(accuracy))
print('Precision: ', np.mean(precision))
print('Recall: ', np.mean(recall))
print('F1: ', np.mean(f1))

('Accuracy: ', 0.79069767441860461)
('Precision: ', 0.25)
('Recall: ', 0.39999999999999991)
('F1: ', 0.3076923076923076)
